In [ ]:
## Get Data Imports
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import requests

## Kafka Imports
import json
from json import dumps
from kafka import KafkaProducer
from time import sleep

In [ ]:
## Global Variables
URL = 'http://transitdata.cityofmadison.com/Vehicle/VehiclePositions.pb'
KAFKA_ADDRESS = '10.128.0.3' ## Internal IP of KafkaVM
KAFKA_TOPIC = 'VehiclePositions'

In [ ]:
## Get Feed from URL
FEED = gtfs_realtime_pb2.FeedMessage()
RESPONSE = requests.get(URL)

In [ ]:
## Kafka Producer
producer = KafkaProducer(bootstrap_servers = [KAFKA_ADDRESS + ":9092"],
                      sasl_mechanism = 'PLAIN',
                      sasl_plain_username = 'admin',
                      sasl_plain_password = '2d15dc73ddb202b5bfa5f451bd684ffc25bf651a987cbe6a0f541239e94377db',
                      security_protocol = 'SASL_PLAINTEXT',
                      api_version=(0,11,5),
                      key_serializer = lambda x: dumps(x).encode('utf-8'),
                      value_serializer = lambda x: dumps(x).encode('utf-8'),
                      acks = 1, ## Default -> Broker Recieve Ack
                      batch_size = 16384, ## Default Value
                      compression_type = 'gzip')

## Parse Response String
FEED.ParseFromString(RESPONSE.content)

## Send to Kafka Topic
for idx, elem in enumerate(FEED.entity):
    print("sending index:", idx)
    producer.send(KAFKA_TOPIC, key= {"id": elem.id}, value=MessageToDict(elem)).get(timeout=30)

In [ ]:
MessageToDict(FEED.entity[-1])